In [1]:
import numpy as np
import qiskit
from qiskit.visualization import circuit_drawer
from qiskit.quantum_info import Statevector
from matplotlib.pyplot import figure
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer, assemble
from qiskit.providers.aer.noise import NoiseModel
import pylatexenc
from qiskit.algorithms import VQE
from qiskit.utils import QuantumInstance
from qiskit.opflow import X, Z, I, Y
from qiskit.circuit import Parameter
from qiskit.algorithms.optimizers import ADAM
from qiskit.opflow import AerPauliExpectation
from qiskit import IBMQ
import cmath
import pandas as pd
from scipy.sparse import diags
import numpy.linalg as LA
import matplotlib.pyplot as plt
from IPython.display import Image
import warnings
warnings.filterwarnings('ignore')
pi=np.pi

In [23]:
eps, V = 2, 0.5
H_exact = np.zeros((3,3))
H_exact[0,0] = -eps
H_exact[2,2] = eps
H_exact[2,0] = H_exact[0,2] = -V

E, C = np.linalg.eigh(H_exact)
E[0] # Ground state to recreate

-2.0615528128088303

In [16]:
def LM_circuit():
    theta = Parameter('theta')
    QC = QuantumCircuit(2)
    QC.ry(theta,0) # 2*(theta-np.pi/2)
    QC.cnot(0,1)
    return QC


adam = qiskit.algorithms.optimizers.ADAM(maxiter=10000)

H = 0.5 * eps * ( Z ^ I ) +0.5 * eps * ( I ^ Z )-0.5 * V * ( X ^ X ) + 0.5 * V *  ( Y ^ Y )
vec = Aer.get_backend('statevector_simulator')
vqe = VQE(ansatz=LM_circuit(),optimizer=adam,quantum_instance=vec,expectation=AerPauliExpectation())
result = vqe.compute_minimum_eigenvalue(H)

result.eigenvalue

(-2.0615528127586717+0j)

In [26]:
eps, V = 2, 0.5
H_exact = np.zeros((5,5))
H_exact[0,0] = H_exact[1,1] = -eps
H_exact[3,3] = H_exact[4,4] = -eps

H_exact[2,0] = H_exact[0,2] = H_exact[2,4] = H_exact[4,2] = np.sqrt(6)*V
H_exact[3,1] = H_exact[1,3] = 3*V

E, C = np.linalg.eigh(H_exact)
E[0] # Ground state to recreate

-3.5

In [32]:
def LM_circuit():
    theta = Parameter('theta')
    QC = QuantumCircuit(2)
    QC.ry(theta,0) # 2*(theta-np.pi/2)
    QC.cnot(0,1)
    return QC

adam = qiskit.algorithms.optimizers.ADAM(maxiter=10000)

H = -(Z ^ I + I ^ Z) - V * (X ^ I + I ^ X + Z ^ X - X ^ Z)
# H = qiskit.Operator(H)
vec = Aer.get_backend('statevector_simulator')
vqe = VQE(ansatz=LM_circuit(),optimizer=adam,quantum_instance=vec,expectation=AerPauliExpectation())
result = vqe.compute_minimum_eigenvalue(H)

result.eigenvalue

ValueError: Sum over operators with different numbers of qubits, 3 and 5, is not well defined